In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import nltk
import spacy
import string

In [10]:
from nltk.corpus import stopwords

## Loading Data

In [4]:
df = pd.read_csv("../data/intermediate/food_reviews.csv")

## Casing

**Tradeoffs** -
1. Difference in sentiment
2. Lesser time & compute

In [5]:
df["Summary"] = df["Summary"].str.lower()

## Puncutations Removal

In [8]:
PUNCT_TO_REMOVE = '"#$%&\'*+,-./<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [9]:
df["Summary"] = df["Summary"].apply(lambda text: remove_punctuation(text))

## Removal of stopwords

In [11]:
STOPWORDS = set(stopwords.words('english'))

In [13]:
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [14]:
df["Summary"] = df["Summary"].apply(lambda text: remove_stopwords(text))

## Remove Frequent Words?

Nope for summary. Revisit the unigram & bigram analysis for summary

## Remove Rare Words?
Nope for summary. Yes for Text

## Lemmatization
Yes for both

## Conversion of Emoticon to Words
Yes for both

## Removal of URLs

## Chat Words Conversion